## Imports

In [1]:
import pandas as pd
import sys

from copy import deepcopy
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(10**4)

In [3]:
day = "19"

In [4]:
# with open(f"input_j{day}.txt") as f:
with open("test.txt") as f:
    donnees = f.readlines()
    donnees = list(map(lambda x: x.replace("\n", ""), donnees))

n = len(donnees)

## Part 1

In [5]:
workflows = {}

i = 0
while donnees[i] != '':
    titre = donnees[i].split('{')[0]
    workflow = donnees[i].split('{')[1][:-1].split(',')
    for k in range(len(workflow)):
        if ':' in workflow[k]:
            workflow[k] = [workflow[k][0], workflow[k][1], int(workflow[k].split(":")[0][2:]), workflow[k].split(":")[1]]
        else:
            workflow[k] = [workflow[k]]
    workflows[titre] = workflow
    i += 1

i += 1

len(workflows)

11

In [6]:
workflows['in']

[['s', '<', 1351, 'px'], ['qqz']]

In [7]:
ratings = pd.DataFrame(columns=['x', 'm', 'a', 's'])

while i < n:
    ratings.loc[len(ratings)] = [int(number.split("=")[1]) for number in donnees[i][1:-1].split(",")]
    i += 1

nb_ratings = len(ratings)

In [8]:
ratings.head(2)

,x,m,a,s
0,787,2655,1222,2876
1,1679,44,2067,496


In [9]:
def is_accepted(row, workflow_name='in'):
    workflow = workflows[workflow_name]
    k = 0
    while len(workflow[k]) != 1:
        if workflow[k][1] == "<" and row[workflow[k][0]] < workflow[k][2]:
            break
        if workflow[k][1] == ">" and row[workflow[k][0]] > workflow[k][2]:
            break
        k += 1
    if workflow[k][-1] in ["A", "R"]:
        return workflow[k][-1]
    return is_accepted(row, workflow[k][-1])


In [10]:
ratings["accepted"] = ratings.apply(is_accepted, axis=1)

In [11]:
ratings.head(2)

,x,m,a,s,accepted
0,787,2655,1222,2876,A
1,1679,44,2067,496,R


In [12]:
accepted_ratings = ratings[ratings["accepted"] == "A"][['x', 'm', 'a', 's']]
accepted_ratings.sum(axis=1).sum()

19114

## Part 2

In [13]:
min_accepted = 1
max_accepted = 4000

### Infructueux

In [14]:
bords = {
    'x': [min_accepted, max_accepted],
    'm': [min_accepted, max_accepted],
    'a': [min_accepted, max_accepted],
    's': [min_accepted, max_accepted]
}

for workflow in workflows.values():
    for condition in workflow:
        if len(condition) > 1:
            nombre = int(condition[2])
            if condition[1] == "<":
                bords[condition[0]].extend([nombre-1, nombre])
            if condition[1] == ">":
                bords[condition[0]].extend([nombre, nombre+1])

for key in bords.keys():
    bords[key].sort()
    print(len(bords[key]))
    

8
10
8
10


In [15]:
# Trop long

"""
new_ratings = pd.DataFrame(columns=['x', 'm', 'a', 's'])
for x in tqdm(range(len(bords['x'])-1)):
    for m in range(len(bords['m'])-1):
        for a in range(len(bords['a'])-1):
            for s in range(len(bords['s'])-1):
                new_ratings.loc[len(new_ratings)] = [
                    bords['x'][x:x+2], bords['m'][m:m+2], bords['a'][a:a+2], bords['s'][s:s+2]
                ]

len(new_ratings)
"""

"\nnew_ratings = pd.DataFrame(columns=['x', 'm', 'a', 's'])\nfor x in tqdm(range(len(bords['x'])-1)):\n    for m in range(len(bords['m'])-1):\n        for a in range(len(bords['a'])-1):\n            for s in range(len(bords['s'])-1):\n                new_ratings.loc[len(new_ratings)] = [\n                    bords['x'][x:x+2], bords['m'][m:m+2], bords['a'][a:a+2], bords['s'][s:s+2]\n                ]\n\nlen(new_ratings)\n"

### Idée J5 - Partie 2

In [16]:
new_ratings = pd.DataFrame(columns=['x', 'm', 'a', 's'])
new_ratings.loc[0] = [
    [min_accepted, max_accepted],
    [min_accepted, max_accepted],
    [min_accepted, max_accepted],
    [min_accepted, max_accepted]
]

In [17]:
new_ratings

,x,m,a,s
0,"[1, 4000]","[1, 4000]","[1, 4000]","[1, 4000]"


In [18]:
def creer_tranches(row, condition):
        j = len(new_ratings)
        if condition[1] == "<":
            if (
                new_ratings.loc[row, condition[0]][0] < condition[2]
            ) and (
                condition[2] <= new_ratings.loc[row, condition[0]][1]
            ):
                new_ratings.loc[j] = new_ratings.loc[row]
                new_ratings.loc[j, condition[0]] = [condition[2], new_ratings.loc[row, condition[0]][1]]
                new_ratings.loc[row, condition[0]] = [new_ratings.loc[row, condition[0]][0], condition[2]-1]
        if condition[1] == ">":
            if (
                new_ratings.loc[row, condition[0]][0] <= condition[2]
            ) and (
                condition[2] < new_ratings.loc[row, condition[0]][1]
            ):
                new_ratings.loc[j] = new_ratings.loc[row]
                new_ratings.loc[j, condition[0]] = [condition[2]+1, new_ratings.loc[row, condition[0]][1]]
                new_ratings.loc[row, condition[0]] = [new_ratings.loc[row, condition[0]][0], condition[2]]

In [19]:
def complete_new_ratings():
    for workflow in workflows.values():
        for condition in workflow:
            if len(condition)>1:
                k = len(new_ratings)
                for i in range(k):
                    creer_tranches(i, condition)

In [20]:
complete_new_ratings()
len(new_ratings)

400

In [21]:
def is_accepted_part_2(row, workflow_name='in'):
    workflow = workflows[workflow_name]
    k = 0
    while len(workflow[k]) != 1:
        if workflow[k][1] == "<" and row[workflow[k][0]][0] < workflow[k][2]:
            break
        if workflow[k][1] == ">" and row[workflow[k][0]][0]  > workflow[k][2]:
            break
        k += 1
    if workflow[k][-1] in ["A", "R"]:
        return workflow[k][-1]
    return is_accepted_part_2(row, workflow[k][-1])


In [22]:
new_ratings["accepted"] = new_ratings.apply(is_accepted_part_2, axis=1)

In [23]:
new_accepted_ratings = new_ratings[new_ratings["accepted"] == "A"][['x', 'm', 'a', 's']]

for col in new_accepted_ratings.columns:
    new_accepted_ratings[col] = new_accepted_ratings[col].apply(lambda x: x[1] - x[0] + 1)

nb_accepted_ratings = new_accepted_ratings.product(axis=1).sum()

In [24]:
nb_accepted_ratings

167409079868000